In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#Load Model
model = 'models/classification_models/inception_resnet_v2-aug_no-normals'
model_name = 'weights-improvement-32-0.89.hdf5'
             
model_dir = os.path.join(model,model_name)

trained_lesion_class_model = load_model(model_dir)

In [3]:
train_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/train_set_class_nn.pickle')
val_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/val_set_class_nn.pickle')

In [4]:
val_set.head()

,image_path,images,lesion_state,lesion_state_one_hot
474,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (345).png,benign,"[1, 0]"
703,/external_drive/BUS_Deep_Learning/data/Dataset...,malignant (170).png,malignant,"[0, 1]"
278,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (278).png,benign,"[1, 0]"
299,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (304).png,benign,"[1, 0]"
632,/external_drive/BUS_Deep_Learning/data/Dataset...,malignant (61).png,malignant,"[0, 1]"


In [5]:
classes = ['benign','malignant']
gts = []
preds = [] 

for index, row in val_set.iterrows():
    print('loading sequence : {}'.format(row['image_path']))
    img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128,128))
    img = img/255.0
    img = np.array([img.reshape(img.shape+(1,))])
    pred = trained_lesion_class_model.predict(img)
    preds.append(classes[np.where(pred[0]==max(pred[0]))[0][0]])
    gts.append(row['lesion_state'])

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (345).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (170).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (278).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (304).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (61).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (405).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (291).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (33).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (13).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Datase

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (42).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (412).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (289).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (32).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (301).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (427).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (14).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (191).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (206).png
loading sequence : /external_drive/BUS_Deep_Learning/data/

In [6]:
results = pd.DataFrame(columns=['prediction','ground_truth'])
results['prediction'] = preds
results['ground_truth'] = gts

In [7]:
#accuracy
len(results.loc[results['prediction']==results['ground_truth']])/len(results)

0.8923076923076924